#Linear regression using PyTorch built-ins

##Define DataLoader

In [4]:
import numpy as np
import torch

In [12]:
# Targets (apples, oranges)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [11]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [14]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


In [7]:
from torch.utils.data import TensorDataset

In [15]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [16]:
from torch.utils.data import DataLoader

In [17]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [18]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]])
tensor([[ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [ 81., 101.],
        [ 22.,  37.]])


##nn.Linear

In [19]:
import torch.nn as nn

In [20]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3723, -0.2983, -0.0499],
        [ 0.5711,  0.1598,  0.3870]], requires_grad=True)
Parameter containing:
tensor([0.1427, 0.0071], requires_grad=True)


In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3723, -0.2983, -0.0499],
         [ 0.5711,  0.1598,  0.3870]], requires_grad=True),
 Parameter containing:
 tensor([0.1427, 0.0071], requires_grad=True)]

In [22]:
preds = model(inputs)
preds

tensor([[  5.1897,  69.0484],
        [  4.5788,  90.8122],
        [-10.3329,  93.5587],
        [ 23.4464,  79.4517],
        [ -6.2987,  81.8490]], grad_fn=<AddmmBackward0>)

##loss functions

In [23]:
import torch.nn.functional as F

In [24]:
loss_fn = F.mse_loss

In [32]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(296.3472, grad_fn=<MseLossBackward0>)


##Optimizer

In [26]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

##Train for multiple epochs

In [27]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [28]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1002.8874
Epoch [20/100], Loss: 806.6610
Epoch [30/100], Loss: 710.6458
Epoch [40/100], Loss: 627.5411
Epoch [50/100], Loss: 554.3514
Epoch [60/100], Loss: 489.8662
Epoch [70/100], Loss: 433.0467
Epoch [80/100], Loss: 382.9787
Epoch [90/100], Loss: 338.8569
Epoch [100/100], Loss: 299.9723


In [29]:
prediction = model(inputs)
prediction

tensor([[ 63.2989,  75.2774],
        [ 84.0186, 101.5356],
        [104.7123, 123.0028],
        [ 56.3571,  65.6361],
        [ 84.5149, 103.9424]], grad_fn=<AddmmBackward0>)

In [30]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [31]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[62.0236, 74.0284]], grad_fn=<AddmmBackward0>)